# Ensuring Consistency with ACID Transactions with Delta Lake (Loan Risk Data)

<img src="https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-logo-whitebackground.png" width=200/>

- Lending Club 데이터에 대한 Delta Lake 예시임
- 튜토리얼은 *DBR 5.4 ML Beta, Python 3*로 테스트

## The Data
- 사용하는 데이터는 Lending Club의 공개된 데이터. 2012 ~ 2017까지의 모든 자금 대출이 포함되어 있음. 각 대출에는 지원자가 제공한 지원자 정보와 현재 대출 상태(현재, 지연, 전액 지불 등), 최신 지불 정보가 포함되어 있음
- 전체 데이터 : [here](https://resources.lendingclub.com/LCDataDictionary.xlsx).

![Loan_Data](https://preview.ibb.co/d3tQ4R/Screen_Shot_2018_02_02_at_11_21_51_PM.png)

https://www.kaggle.com/wendykan/lending-club-loan-data

## ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Delta Lake

- Optimization Layer a top blob storage for Reliability (i.e. ACID compliance) and Low Latency of Streaming + Batch data pipelines.

## Import Data and create pre-Delta Lake Table

In [5]:
# 스파크 분산처리를 위한 파티션의 갯수를 설정
sqlContext.setConf("spark.sql.shuffle.partitions", "2")

In [6]:
# -----------------------------------------------
# Uncomment and run if this folder does not exist
# -----------------------------------------------

# Configure location of loanstats_2012_2017.parquet
lspq_path = "/databricks-datasets/samples/lending_club/parquet/"

# loanstats_2012_2017.parquet 을 읽어드림
data = spark.read.parquet(lspq_path)

# 실험을 위하여 데이터의 개수를 줄이는 작업(to run on DBCE)
(loan_stats, loan_stats_rest) = data.randomSplit([0.10, 0.90], seed=123)

loan_stats_rest.createOrReplaceTempView("loan_data_full")

# 필요한 컬럼만 선택
loan_stats = loan_stats.select("addr_state", "loan_status")

# state(미국의 주)별로 group by하여 transaction을 카운트
loan_by_state = loan_stats.groupBy("addr_state").count()

# 결과물을 TempView 테이블로 생성
loan_by_state.createOrReplaceTempView("loan_by_state")

# 확인
display(loan_by_state)

addr_state,count
NY,12003
AZ,3531
MD,3544
TN,2313
IL,5978
VA,4233
NJ,5461
CT,2243
NC,4176
SC,1797


## ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Parquet을 Delta Lake 형식으로 변환
- Delta Lake를 사용하면 Parquet 데이터를 Delta Lake 형식으로 쉽게 변경할 수 있음

In [8]:
# Delta Lake Silver Path를 설정
DELTALAKE_SILVER_PATH = "/ml/loan_by_state_delta"

# 만약 데이터가 존재한다면 제거
dbutils.fs.rm(DELTALAKE_SILVER_PATH, recurse=True)

Out[3]: True

In [9]:
%sql 
-- 설정한 경로에 새로운 새로운 테이블을 생성(델타 테이블을 생성하는 것)
DROP TABLE IF EXISTS loan_by_state_delta;

CREATE TABLE loan_by_state_delta
USING delta
LOCATION '/ml/loan_by_state_delta'
AS SELECT * FROM loan_by_state;

In [10]:
%sql
-- View Delta Lake table
SELECT * FROM loan_by_state_delta

addr_state,count
OH,5108
CA,20461
TX,12348
FL,10455
CO,3143
WA,3141
LA,1792
AL,1829
UT,979
AR,1163


In [11]:
%sql 
-- delta 형식으로 저장한 경로의 detail을 describe
-- .``을 사용해서 경로를 설정해야 함
DESCRIBE DETAIL delta .`/ml/loan_by_state_delta`

format,id,name,description,location,createdAt,lastModified,partitionColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion
delta,5b588d6a-cae1-444a-8a12-36531ce7c74b,null,null,dbfs:/ml/loan_by_state_delta,2020-08-26T20:50:51.637+0000,2020-08-26T20:51:17.000+0000,List(),2,1861,Map(),1,2


## Stop the notebook before the streaming cell, in case of a "run all"

In [13]:
dbutils.notebook.exit("stop") 

stop

In [14]:
%fs ls /ml/loan_by_state_delta/_delta_log/

path,name,size
dbfs:/ml/loan_by_state_delta/_delta_log/.s3-optimization-0,.s3-optimization-0,0
dbfs:/ml/loan_by_state_delta/_delta_log/.s3-optimization-1,.s3-optimization-1,0
dbfs:/ml/loan_by_state_delta/_delta_log/.s3-optimization-2,.s3-optimization-2,0
dbfs:/ml/loan_by_state_delta/_delta_log/00000000000000000000.crc,00000000000000000000.crc,89
dbfs:/ml/loan_by_state_delta/_delta_log/00000000000000000000.json,00000000000000000000.json,1590


## ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Batch와 Streaming의 통합과 Sink

- 이 셀들은 스트리밍과 배치의 concurrent 쿼리(insert, read)를 보여줌
* 이 노트북은 `loan_stats_delta` 테이블에 대해서 10초 마다 `INSERT`를 실행함
* 이 데이터에 대해서 2개의 스트리밍 쿼리를 concurrent하게 실행
* 참고 : `writeStream`도 사용할 수 있지만 DBCE에서는 이러한 접근이 더 쉬움

In [16]:
# Read the insertion of data
loan_by_state_readStream = spark.readStream.format("delta").load(DELTALAKE_SILVER_PATH)
loan_by_state_readStream.createOrReplaceTempView("loan_by_state_readStream")

In [17]:
%sql
select addr_state, sum(`count`) as loans from loan_by_state_readStream group by addr_state

addr_state,loans
IL,5978
MI,3814
SC,1797
OR,1692
ID,168
AZ,3531
IA,27000
null,1
MN,2724
SD,312


- 아래의 코드를 실행하기 전에 스트림이 실행될 때까지 기다려야함

In [19]:
import time
i = 1
while i <= 6:
  # Execute Insert statement
  insert_sql = "INSERT INTO loan_by_state_delta VALUES ('IA', 4500)"
  spark.sql(insert_sql)
  print('loan_by_state_delta: inserted new row of data, loop: [%s]' % i)
    
  # Loop through
  i = i + 1
  time.sleep(5)

loan_by_state_delta: inserted new row of data, loop: [1]
loan_by_state_delta: inserted new row of data, loop: [2]
loan_by_state_delta: inserted new row of data, loop: [3]
loan_by_state_delta: inserted new row of data, loop: [4]
loan_by_state_delta: inserted new row of data, loop: [5]
loan_by_state_delta: inserted new row of data, loop: [6]

- 시간이 자나면서 위에 IW에 해당하는 값이 점점 증가하고 map의 색상이 짙어지는 것을 알 수 있음

In [21]:
%fs ls /ml/loan_by_state_delta/_delta_log/

path,name,size
dbfs:/ml/loan_by_state_delta/_delta_log/.s3-optimization-0,.s3-optimization-0,0
dbfs:/ml/loan_by_state_delta/_delta_log/.s3-optimization-1,.s3-optimization-1,0
dbfs:/ml/loan_by_state_delta/_delta_log/.s3-optimization-2,.s3-optimization-2,0
dbfs:/ml/loan_by_state_delta/_delta_log/00000000000000000000.crc,00000000000000000000.crc,89
dbfs:/ml/loan_by_state_delta/_delta_log/00000000000000000000.json,00000000000000000000.json,1590
dbfs:/ml/loan_by_state_delta/_delta_log/00000000000000000001.crc,00000000000000000001.crc,89
dbfs:/ml/loan_by_state_delta/_delta_log/00000000000000000001.json,00000000000000000001.json,761
dbfs:/ml/loan_by_state_delta/_delta_log/00000000000000000002.crc,00000000000000000002.crc,89
dbfs:/ml/loan_by_state_delta/_delta_log/00000000000000000002.json,00000000000000000002.json,761
dbfs:/ml/loan_by_state_delta/_delta_log/00000000000000000003.crc,00000000000000000003.crc,89


In [22]:
%sql 

Describe history loan_by_state_delta

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics
6,2020-08-26T20:54:21.000+0000,4425136926506324,suhoy90@naver.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(904961833301219),0826-191813-loss58,5,WriteSerializable,true,"Map(numFiles -> 1, numOutputBytes -> 703, numOutputRows -> 1)"
5,2020-08-26T20:54:14.000+0000,4425136926506324,suhoy90@naver.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(904961833301219),0826-191813-loss58,4,WriteSerializable,true,"Map(numFiles -> 1, numOutputBytes -> 703, numOutputRows -> 1)"
4,2020-08-26T20:54:07.000+0000,4425136926506324,suhoy90@naver.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(904961833301219),0826-191813-loss58,3,WriteSerializable,true,"Map(numFiles -> 1, numOutputBytes -> 703, numOutputRows -> 1)"
3,2020-08-26T20:53:59.000+0000,4425136926506324,suhoy90@naver.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(904961833301219),0826-191813-loss58,2,WriteSerializable,true,"Map(numFiles -> 1, numOutputBytes -> 703, numOutputRows -> 1)"
2,2020-08-26T20:53:51.000+0000,4425136926506324,suhoy90@naver.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(904961833301219),0826-191813-loss58,1,WriteSerializable,true,"Map(numFiles -> 1, numOutputBytes -> 703, numOutputRows -> 1)"
1,2020-08-26T20:53:44.000+0000,4425136926506324,suhoy90@naver.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(904961833301219),0826-191813-loss58,0,WriteSerializable,true,"Map(numFiles -> 1, numOutputBytes -> 703, numOutputRows -> 1)"
0,2020-08-26T20:51:17.000+0000,4425136926506324,suhoy90@naver.com,CREATE TABLE AS SELECT,"Map(isManaged -> false, description -> null, partitionBy -> [], properties -> {})",null,List(904961833301219),0826-191813-loss58,null,WriteSerializable,true,"Map(numFiles -> 2, numOutputBytes -> 1861, numOutputRows -> 51)"


- **Note**: 이전 셀이 완료되고 lowa 상태가 맵(15)에 완전하게 채워지면 셀 15에서 취소를 클릭해서 readStream을 중지

- 지도 시각화를 사용해서 현재의 대출 상태를 검토

In [25]:
%sql
-- Review current loans within the `loan_by_state_delta` Delta Lake table
select addr_state, sum(`count`) as loans from loan_by_state_delta group by addr_state

addr_state,loans
NY,12003
AZ,3531
MD,3544
TN,2313
IL,5978
VA,4233
NJ,5461
CT,2243
NC,4176
SC,1797


- 최근 데이터 흐름으로 인해 아이오와에서 대출 건수가 가장 많음을 확인할 수 있음
  - `loan_by_state_readStream`을 읽는 동안 원본인 `loan_by_state_delta` 테이블이 업데이트 됨

##![Delta Lake Logo Tiny](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Full DML Support

- **Note**: Delta Lake에 완전한 DML이 지원될 예정임. 현재는 Databricks에서 preview로 사용할 수 있음
- Delta Lake는 UPDATE, DELETE, MERGE, INTO 를 포함한 표준 DML을 지원하기 때문에 데이터엔지니어들이 대량의 데이터를 관리하기 위해 제어를 할 수 있게 도와줌

- 먼저 전통적인 Parquet 테이블을 생성

In [29]:
# Load new DataFrame based on current Delta table
lbs_df = sql("select * from loan_by_state_delta")

# Save DataFrame to Parquet
lbs_df.write.mode("overwrite").parquet("/loan_by_state.parquet")

# Reload Parquet Data
lbs_pq = spark.read.parquet("/loan_by_state.parquet")

# Create new table on this parquet data
lbs_pq.createOrReplaceTempView("loan_by_state_pq")

# Review data
display(sql("select * from loan_by_state_pq"))

addr_state,count
OH,5108
CA,20461
TX,12348
FL,10455
CO,3143
WA,3141
LA,1792
AL,1829
UT,979
AR,1163


###![Delta Lake Logo Tiny](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) DELETE Support

- 데이터는 원래 `WA` 주에 할당되어야하기 때문에 `IA`에 할당된 값을 `DELETE`

In [31]:
%sql
-- Attempting to run `DELETE` on the Parquet table
DELETE FROM loan_by_state_pq WHERE addr_state = 'IA'

com.databricks.backend.common.rpc.DatabricksExceptions$SQLExecutionException: org.apache.spark.sql.AnalysisException: Invalid command: 'loan_by_state_pq' is a view not a table.; line 1 pos 0;
'DeleteFromTable ('addr_state = IA)
+- 'UnresolvedV2Relation [loan_by_state_pq], com.databricks.sql.transaction.tahoe.catalog.DeltaCatalog@7de895dc, default.loan_by_state_pq

	at org.apache.spark.sql.catalyst.analysis.package$AnalysisErrorAt.failAnalysis(package.scala:42)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.$anonfun$checkAnalysis$1(CheckAnalysis.scala:116)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.$anonfun$checkAnalysis$1$adapted(CheckAnalysis.scala:95)
	at org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.scala:184)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1(TreeNode.scala:183)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1$adapted(TreeNode.scala:183)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.scala:183)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.checkAnalysis(CheckAnalysis.scala:95)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.checkAnalysis$(CheckAnalysis.scala:92)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.checkAnalysis(Analyzer.scala:136)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.$anonfun$executeAndCheck$1(Analyzer.scala:164)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.markInAnalyzer(AnalysisHelper.scala:201)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.executeAndCheck(Analyzer.scala:161)
	at org.apache.spark.sql.execution.QueryExecution.$anonfun$analyzed$1(QueryExecution.scala:90)
	at org.apache.spark.sql.catalyst.QueryPlanningTracker.measurePhase(QueryPlanningTracker.scala:111)
	at org.apache.spark.sql.execution.QueryExecution.$anonfun$executePhase$1(QueryExecution.scala:162)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:828)
	at org.apache.spark.sql.execution.QueryExecution.executePhase(QueryExecution.scala:162)
	at org.apache.spark.sql.execution.QueryExecution.analyzed$lzycompute(QueryExecution.scala:90)
	at org.apache.spark.sql.execution.QueryExecution.analyzed(QueryExecution.scala:88)
	at org.apache.spark.sql.execution.QueryExecution.assertAnalyzed(QueryExecution.scala:80)
	at org.apache.spark.sql.Dataset$.$anonfun$ofRows$2(Dataset.scala:102)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:828)
	at org.apache.spark.sql.Dataset$.ofRows(Dataset.scala:100)
	at org.apache.spark.sql.SparkSession.$anonfun$sql$1(SparkSession.scala:663)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:828)
	at org.apache.spark.sql.SparkSession.sql(SparkSession.scala:658)
	at org.apache.spark.sql.SQLContext.sql(SQLContext.scala:672)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.$anonfun$executeSql$1(SQLDriverLocal.scala:88)
	at scala.collection.TraversableLike.$anonfun$map$1(TraversableLike.scala:238)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at scala.collection.TraversableLike.map(TraversableLike.scala:238)
	at scala.collection.TraversableLike.map$(TraversableLike.scala:231)
	at scala.collection.immutable.List.map(List.scala:298)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.executeSql(SQLDriverLocal.scala:34)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.repl(SQLDriverLocal.scala:141)
	at com.databricks.backend.daemon.driver.DriverLocal.$anonfun$execute$10(DriverLocal.scala:396)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:238)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:233)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:230)
	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionContext(Drive

**Note**: 위 명령은 parquet에서 `DELETE`을 지원하지 않기 때문에 실패함. 하지만 Delta Lake에서는 지원하기 때문에 아래의 명령어는 성공하게 됨

In [33]:
%sql
-- Running `DELETE` on the Delta Lake table
DELETE FROM loan_by_state_delta WHERE addr_state = 'IA'

In [34]:
%sql
-- Review current loans within the `loan_by_state_delta` Delta Lake table
select addr_state, sum(`count`) as loans from loan_by_state_delta group by addr_state

addr_state,loans
NY,12003
AZ,3531
MD,3544
TN,2313
IL,5978
VA,4233
NJ,5461
CT,2243
NC,4176
SC,1797


- Delta 테이블에서 작업을 수행한 결과 아이오와 주의 데이터들이 사라진 것을 알 수 있음

###![Delta Lake Logo Tiny](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) UPDATE Support
- 원 데이터가 `WA` 주에 할당되어야 하므로 해강 값을 다시 `UPDATE`

In [37]:
%sql
-- Attempting to run `UPDATE` on the Parquet table
UPDATE loan_by_state_pq SET `count` = 27000 WHERE addr_state = 'WA'

com.databricks.backend.common.rpc.DatabricksExceptions$SQLExecutionException: org.apache.spark.sql.AnalysisException: UPDATE destination only supports Delta sources.
Some(Relation[addr_state#26525,count#26526L] parquet
);
	at com.databricks.sql.transaction.tahoe.DeltaErrors$.notADeltaSourceException(DeltaErrors.scala:246)
	at com.databricks.sql.transaction.tahoe.PreprocessTableUpdateEdge$$anonfun$apply0$1.applyOrElse(PreprocessTableUpdateEdge.scala:56)
	at com.databricks.sql.transaction.tahoe.PreprocessTableUpdateEdge$$anonfun$apply0$1.applyOrElse(PreprocessTableUpdateEdge.scala:50)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.$anonfun$resolveOperatorsDown$2(AnalysisHelper.scala:108)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:79)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.$anonfun$resolveOperatorsDown$1(AnalysisHelper.scala:108)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.allowInvokingTransformsInAnalyzer(AnalysisHelper.scala:194)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperatorsDown(AnalysisHelper.scala:106)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperatorsDown$(AnalysisHelper.scala:104)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveOperatorsDown(LogicalPlan.scala:29)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperators(AnalysisHelper.scala:73)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperators$(AnalysisHelper.scala:72)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveOperators(LogicalPlan.scala:29)
	at com.databricks.sql.transaction.tahoe.PreprocessTableUpdateEdge.apply0(PreprocessTableUpdateEdge.scala:50)
	at com.databricks.sql.transaction.tahoe.PreprocessTableUpdateEdge.apply(PreprocessTableUpdateEdge.scala:46)
	at com.databricks.sql.transaction.tahoe.PreprocessTableUpdateEdge.apply(PreprocessTableUpdateEdge.scala:39)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.$anonfun$execute$2(RuleExecutor.scala:152)
	at scala.collection.IndexedSeqOptimized.foldLeft(IndexedSeqOptimized.scala:60)
	at scala.collection.IndexedSeqOptimized.foldLeft$(IndexedSeqOptimized.scala:68)
	at scala.collection.mutable.ArrayBuffer.foldLeft(ArrayBuffer.scala:49)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.$anonfun$execute$1(RuleExecutor.scala:149)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.$anonfun$execute$1$adapted(RuleExecutor.scala:141)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.execute(RuleExecutor.scala:141)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.org$apache$spark$sql$catalyst$analysis$Analyzer$$executeSameContext(Analyzer.scala:184)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.execute(Analyzer.scala:178)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.execute(Analyzer.scala:136)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.$anonfun$executeAndTrack$1(RuleExecutor.scala:119)
	at org.apache.spark.sql.catalyst.QueryPlanningTracker$.withTracker(QueryPlanningTracker.scala:88)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.executeAndTrack(RuleExecutor.scala:119)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.$anonfun$executeAndCheck$1(Analyzer.scala:162)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.markInAnalyzer(AnalysisHelper.scala:201)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.executeAndCheck(Analyzer.scala:161)
	at org.apache.spark.sql.execution.QueryExecution.$anonfun$analyzed$1(QueryExecution.scala:90)
	at org.apache.spark.sql.catalyst.QueryPlanningTracker.measurePhase(QueryPlanningTracker.scala:111)
	at org.apache.spark.sql.execution.QueryExecution.$anonfun$executePhase$1(QueryExecution.scala:162)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:828)
	at org.apache.spark.sql.execution.QueryExecution.executePhase(Quer

**Note**: parquet에서 `UPDATE`를 지원하지 않기 때문에 실행되지 않는 것을 알 수 있지만 Delta Lake에서는 지원되기 때문에 아래의 명령어가 실행되는 것을 알 수 있음

In [39]:
%sql
-- Running `UPDATE` on the Delta Lake table
UPDATE loan_by_state_delta SET `count` = 27000 WHERE addr_state = 'WA'

In [40]:
%sql
-- Review current loans within the `loan_by_state_delta` Delta Lake table
select addr_state, sum(`count`) as loans from loan_by_state_delta group by addr_state

addr_state,loans
NY,12003
AZ,3531
MD,3544
TN,2313
IL,5978
VA,4233
NJ,5461
CT,2243
NC,4176
SC,1797


###![Delta Lake Logo Tiny](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) MERGE INTO Support

#### INSERT or UPDATE parquet: 7단계의 절차

레거시 데이터 파이프라인을 사용하여 테이블을 삽입하거나 업데이트하려면 아래의 절차를 수행해야함

1. Insert할 새로운 행을 식별
2. replace(ex. update)할 행을 식별
3. 삽입 또는 업데이트의 영향을 받지 않는 모든 행을 식별
4. 위의 3가지 삽입 상태를 기반으로 새로운 temp 를 생성
5. 원본 테이블(관련된 모든 파일 포함)을 삭제
6. temp 테이블의 이름을 기존 원본 테이블의 이름으로 다시 이름을 변경
7. temp 테이블 삭제

![](https://pages.databricks.com/rs/094-YMS-629/images/merge-into-legacy.gif)


#### INSERT or UPDATE with Delta Lake : 2단계의 절차
1. 삽입 또는 업데이트할 행을 식별
2. `MERGE`를 사용

In [42]:
# Let's create a simple table to merge
items = [('IA', 1000000), ('CA', 25), ('OR', None)]
cols = ['addr_state', 'count']

merge_table = spark.createDataFrame(items, cols)
merge_table.createOrReplaceTempView("merge_table")
display(merge_table)

addr_state,count
IA,1000000
CA,25
OR,null


별도의 `INSERT`와 `UPDATE`문을 작성하는 대신에 `MERGE`문을 사용할 수 있음

In [44]:
%sql
MERGE INTO loan_by_state_delta as d
USING merge_table as m
on d.addr_state = m.addr_state
WHEN MATCHED THEN 
  UPDATE SET *
WHEN NOT MATCHED 
  THEN INSERT *

In [45]:
%sql
-- Review current loans within the `loan_by_state_delta` Delta Lake table
select addr_state, sum(`count`) as loans from loan_by_state_delta group by addr_state

addr_state,loans
AZ,3531
CT,2243
IA,1000000
ID,168
IL,5978
MD,3544
MI,3814
MN,2724
MO,2388
MS,799


- 변경된 것을 알 수 있음

##![Delta Lake Logo Tiny](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Schema Evolution
`mergeSchema` option을 사용하면 Delta Lake 테이블 스키마를 evolve할 수 있음

In [48]:
# Generate new loans with dollar amounts 
loans = sql("select addr_state, cast(rand(10)*count as bigint) as count, cast(rand(10) * 10000 * count as double) as amount from loan_by_state_delta")
display(loans)

addr_state,count,amount
AK,59,596614.5011464932
AL,1472,1.4725542299191985E7
AR,671,6717401.445573028
CA,23,236901.1967470231
CO,658,6580514.7445245115
DC,132,1323578.436507916
DE,346,3465757.2285215375
FL,7459,7.459792459674548E7
GA,3499,3.499806355451264E7
HI,249,2491155.7387384484


In [49]:
# Let's write this data out to our Delta table
loans.write.format("delta").mode("append").save(DELTALAKE_SILVER_PATH)

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-904961833301264> in <module> 
 1 # Let's write this data out to our Delta table 
 ----> 2 loans . write . format ( "delta" ) . mode ( "append" ) . save ( DELTALAKE_SILVER_PATH ) 

 /databricks/spark/python/pyspark/sql/readwriter.py in save (self, path, format, mode, partitionBy, **options) 
 825 self . _jwrite . save ( ) 
 826 else : 
 --> 827 self . _jwrite . save ( path ) 
 828 
 829 @ since ( 1.4 ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1303 answer = self . gateway_client . send_command ( command ) 
 1304 return_value = get_return_value(
 -> 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 
 1307 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 132 # Hide where the exception came from that shows a non-Pythonic 
 133 # JVM exception message. 
 --> 134 raise_from ( converted ) 
 135 else : 
 136 raise 

 /databricks/spark/python/pyspark/sql/utils.py in raise_from (e) 

 AnalysisException : A schema mismatch detected when writing to the Delta table (Table ID: 5b588d6a-cae1-444a-8a12-36531ce7c74b).
To enable schema migration using DataFrameWriter or DataStreamWriter, please set:
'.option("mergeSchema", "true")'.
For other operations, set the session configuration
spark.databricks.delta.schema.autoMerge.enabled to "true". See the documentation
specific to the operation for details.

Table schema:
root
-- addr_state: string (nullable = true)
-- count: long (nullable = true)


Data schema:
root
-- addr_state: string (nullable = true)
-- count: long (nullable = true)
-- amount: double (nullable = true)

 ;

**Note**: 위 명령어는 새로운 데이터의 스키마가 원본 데이터의 스키마와 일치하지 않기 때문에 실패하는 것을 알 수 있음

In [51]:
# Add the mergeSchema option
loans.write.option("mergeSchema","true").format("delta").mode("append").save(DELTALAKE_SILVER_PATH)

In [52]:
%sql

select * from loan_by_state_delta

addr_state,count,amount
AK,59,596614.5011464932
AL,1472,1.4725542299191985E7
AR,671,6717401.445573028
CA,23,236901.1967470231
CO,658,6580514.7445245115
DC,132,1323578.436507916
DE,346,3465757.2285215375
FL,7459,7.459792459674548E7
GA,3499,3.499806355451264E7
HI,249,2491155.7387384484


**Note**: `mergeSchema` option을 사용하면 서로 다른 스키마를 병합하여 데이터를 유지할 수 있음
- amount라는 새로운 컬럼에 null값들이 존재하는 것을 확인

In [54]:
%sql
-- Review current loans within the `loan_by_state_delta` Delta Lake table
select addr_state, sum(`amount`) as amount from loan_by_state_delta group by addr_state order by sum(`amount`) desc limit 10

addr_state,amount
IA,4.358851046481089E9
WA,1.3705828403273156E8
TX,8.424605679134685E7
FL,7.459792459674548E7
IL,5.142189603610473E7
NJ,4.405643435616983E7
GA,3.499806355451264E7
NC,3.113454727496868E7
MD,2.8598207745760623E7
VA,2.5915123195418544E7


## ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Let's Travel back in Time!

Databricks Delta의 시간 이동 기능은 아래의 사용 사례를 위한 데이터 파이프라인 구축을 단순화할 수 있음
* 데이터 변경(Data Change)에 대한 감사
* experiments & reports의 재현
* Rollbacks

Delta 테이블 또는 디렉토리에 write하는 경우 **모든 작업의 버전**들이 자동으로 생성
아래를 통해서 쿼리할 수 있음
- timestamp 를 사용
- version number를 사용

파이썬, 스칼라 또는 스칼라 구문 사용의 예시 : [Introducing Delta Time Travel for Large Scale Data Lakes](https://databricks.com/blog/2019/02/04/introducing-delta-time-travel-for-large-scale-data-lakes.html)

### ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Review Delta Lake Table History

테이블에 대한 모든 트랙재션은 스키마 수정을 비롯해서 삽입, 업데이트, 삭제, 병합 등을 포함해 테이블 내에 저장됨

In [57]:
%sql
DESCRIBE HISTORY loan_by_state_delta

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics
10,2020-08-26T21:01:36.000+0000,4425136926506324,suhoy90@naver.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(904961833301219),0826-191813-loss58,9,WriteSerializable,false,"Map(numFiles -> 2, numOutputBytes -> 2777, numOutputRows -> 52)"
9,2020-08-26T21:01:24.000+0000,4425136926506324,suhoy90@naver.com,MERGE,Map(predicate -> (d.`addr_state` = m.`addr_state`)),null,List(904961833301219),0826-191813-loss58,8,WriteSerializable,false,"Map(numTargetRowsCopied -> 49, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 2, numTargetRowsInserted -> 1, numTargetRowsUpdated -> 2, numOutputRows -> 52, numSourceRows -> 3, numTargetFilesRemoved -> 2)"
8,2020-08-26T21:01:10.000+0000,4425136926506324,suhoy90@naver.com,UPDATE,Map(predicate -> (addr_state#26922 = WA)),null,List(904961833301219),0826-191813-loss58,7,WriteSerializable,false,"Map(numRemovedFiles -> 1, numAddedFiles -> 1, numUpdatedRows -> 1, numCopiedRows -> 29)"
7,2020-08-26T21:00:52.000+0000,4425136926506324,suhoy90@naver.com,DELETE,"Map(predicate -> [""(`addr_state` = 'IA')""])",null,List(904961833301219),0826-191813-loss58,6,WriteSerializable,false,"Map(numRemovedFiles -> 6, numDeletedRows -> 6, numAddedFiles -> 1, numCopiedRows -> 0)"
6,2020-08-26T20:54:21.000+0000,4425136926506324,suhoy90@naver.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(904961833301219),0826-191813-loss58,5,WriteSerializable,true,"Map(numFiles -> 1, numOutputBytes -> 703, numOutputRows -> 1)"
5,2020-08-26T20:54:14.000+0000,4425136926506324,suhoy90@naver.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(904961833301219),0826-191813-loss58,4,WriteSerializable,true,"Map(numFiles -> 1, numOutputBytes -> 703, numOutputRows -> 1)"
4,2020-08-26T20:54:07.000+0000,4425136926506324,suhoy90@naver.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(904961833301219),0826-191813-loss58,3,WriteSerializable,true,"Map(numFiles -> 1, numOutputBytes -> 703, numOutputRows -> 1)"
3,2020-08-26T20:53:59.000+0000,4425136926506324,suhoy90@naver.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(904961833301219),0826-191813-loss58,2,WriteSerializable,true,"Map(numFiles -> 1, numOutputBytes -> 703, numOutputRows -> 1)"
2,2020-08-26T20:53:51.000+0000,4425136926506324,suhoy90@naver.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(904961833301219),0826-191813-loss58,1,WriteSerializable,true,"Map(numFiles -> 1, numOutputBytes -> 703, numOutputRows -> 1)"
1,2020-08-26T20:53:44.000+0000,4425136926506324,suhoy90@naver.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(904961833301219),0826-191813-loss58,0,WriteSerializable,true,"Map(numFiles -> 1, numOutputBytes -> 703, numOutputRows -> 1)"


- 위의 명령을 실행한 결과를 확인하면 모든 트랜잭션의 history들이 기록되어 있는 것을 알 수 있음

### ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Time Travel via Version Number
아래는 버전 번호를 사용해 Delta Time Travel의 시간 여행의 예시임

In [60]:
%sql
SELECT * FROM loan_by_state_delta VERSION AS OF 0

addr_state,count
OH,5108
CA,20461
TX,12348
FL,10455
CO,3143
WA,3141
LA,1792
AL,1829
UT,979
AR,1163


In [61]:
%sql
SELECT * FROM loan_by_state_delta VERSION AS OF 9

addr_state,count
AK,349
AL,1829
AR,1163
CA,25
CO,3143
DC,361
DE,429
FL,10455
GA,4864
HI,795


### ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Delta Optimizations
Databricks Delta가 성능을 최적화하는 방법

## VACUUM

저장 비용을 절약하기 위해선 가끔씩 `VACUUM` 명령을 사용해서 잘못된 파일을 정리하면 됨

invalid한 파일은 `OPTIMIZE` 명령을 사용해 더 큰 파일로 압축됨

`VACUUM` 명령의 구문은 아래같이 사용하면 됨 
>`VACUUM name-of-table RETAIN number-of HOURS;`

`number-of` 파라미터는 hours 단위로 지정된 retention interval 임

<img alt="Caution" title="Caution" style="vertical-align: text-bottom; position: relative; height:1.3em; top:0.0em" src="https://files.training.databricks.com/static/images/icon-warning.svg"/> 데이터브릭스는 기존 스냅샷과 커밋되지 않은 파일을 테이블에서 concurrent readers 또는 writer가 계속 사용할 수 있기 때문에 보존 간격을 7일 이내로 설정하는 것을 권장하지 않음

시나리오
0. 사용자 A가 압축되지(uncompacted) 않은 파일에 대해 쿼리를 시작한 다음
0. 사용자 B는 `VACUUM`명령을 호출해 압축되지(uncompacted) 않은 파일을 삭제
0. 파일이 사라져 사용자 A의 쿼리라 실패

updates/upserts/deletion 을 통해서 invalid 파일이 발생할 수 있음

자세한 내용은 여기서 확인: <a href="https://docs.databricks.com/delta/optimizations.html#garbage-collection" target="_blank"> Garbage Collection</a>.

vacuum을 하기 전에 파일 수를 확인해야 함

In [64]:
display(dbutils.fs.ls('/ml/loan_by_state_delta'))

path,name,size
dbfs:/ml/loan_by_state_delta/_delta_log/,_delta_log/,0
dbfs:/ml/loan_by_state_delta/part-00000-216e2230-0f6b-4f4d-992f-c55535beb372-c000.snappy.parquet,part-00000-216e2230-0f6b-4f4d-992f-c55535beb372-c000.snappy.parquet,703
dbfs:/ml/loan_by_state_delta/part-00000-3ce01c3a-77e0-47f5-bc77-29a9d7ecdcc2-c000.snappy.parquet,part-00000-3ce01c3a-77e0-47f5-bc77-29a9d7ecdcc2-c000.snappy.parquet,703
dbfs:/ml/loan_by_state_delta/part-00000-41e25450-a6d6-438f-a566-525716628184-c000.snappy.parquet,part-00000-41e25450-a6d6-438f-a566-525716628184-c000.snappy.parquet,892
dbfs:/ml/loan_by_state_delta/part-00000-4966974d-11db-41e2-ae7e-2e51963b1977-c000.snappy.parquet,part-00000-4966974d-11db-41e2-ae7e-2e51963b1977-c000.snappy.parquet,970
dbfs:/ml/loan_by_state_delta/part-00000-58a53ed7-6a9f-44b6-b129-7e076446d17d-c000.snappy.parquet,part-00000-58a53ed7-6a9f-44b6-b129-7e076446d17d-c000.snappy.parquet,703
dbfs:/ml/loan_by_state_delta/part-00000-79fafdfc-dbaf-47fe-bd40-14e2a559708f-c000.snappy.parquet,part-00000-79fafdfc-dbaf-47fe-bd40-14e2a559708f-c000.snappy.parquet,1460
dbfs:/ml/loan_by_state_delta/part-00000-b16ae54e-2a0f-48f8-be4c-581a14da395f-c000.snappy.parquet,part-00000-b16ae54e-2a0f-48f8-be4c-581a14da395f-c000.snappy.parquet,407
dbfs:/ml/loan_by_state_delta/part-00000-c1f86b76-39b3-473b-97c6-a6708cb12afd-c000.snappy.parquet,part-00000-c1f86b76-39b3-473b-97c6-a6708cb12afd-c000.snappy.parquet,703
dbfs:/ml/loan_by_state_delta/part-00000-c476ae6d-a9a3-4554-93ba-af40a704a253-c000.snappy.parquet,part-00000-c476ae6d-a9a3-4554-93ba-af40a704a253-c000.snappy.parquet,703


In [65]:
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", False)

In [66]:
%sql
VACUUM loan_by_state_delta RETAIN 0 HOURS

path
dbfs:/ml/loan_by_state_delta


## Other Optimizations 
### Let's See How Databricks Delta Makes Spark Queries Faster!

데이터브릭스 델타가 쿼리 성능을 최적화할 수 있는 방법을 확인. Parquet 포맷을 사용해 표준 테이블을 만들고 지연 시간을 관찰하기 위해 빠른 질의를 실행. 그 이후 데이터브릭스 델타 버전에 대한 두번째 쿼리를 실행해 표준 테이블과 데이터브릭스 델타 테이블 간의 성능 차이를 확인

Simply follow these 4 steps below:
* __Step 1__ : 미국 시준 비행 스케쥴 데이터로 표준 Parquet 기반 테이블을 작성
* __Step 2__ : 1년간 출발 공항당 월별 항공편 수를 계산하기 위한 쿼리를 작성하고 실행
* __Step 3__ : Databricks Delta를 사용해 flights 테이블을 만들고 테이블을 optimize
* __Step 4__ : 2단계 쿼리를 다시 실행하고 latency 시간을 확인 

**Note**: *10만 개의 행이 있는 몇 개의 테이블을 만들 것이기 때문에 일부 작업은 클러스터 구성에 따라서 몇 분 정도 시간이 걸릴 수 있음*

In [68]:
dbutils.fs.rm("dbfs:/user/hive/warehouse/flights", True)

Out[14]: False

In [69]:
%sql
-- 시간 : 5.74 minutes
DROP TABLE IF EXISTS flights;

-- Create a standard table and import US based flights for year 2008
-- USING Clause: Specify parquet format for a standard table
-- PARTITIONED BY clause: Orginize data based on "Origin" column (Originating Airport code).
-- FROM Clause: Import data from a csv file. 
CREATE TABLE flights
USING parquet
PARTITIONED BY (Origin)
SELECT _c0 as Year, _c1 as Month, _c2 as DayofMonth, _c3 as DayOfWeek, _c4 as DepartureTime, _c5 as CRSDepartureTime, _c6 as ArrivalTime, 
  _c7 as CRSArrivalTime, _c8 as UniqueCarrier, _c9 as FlightNumber, _c10 as TailNumber, _c11 as ActualElapsedTime, _c12 as CRSElapsedTime, 
    _c13 as AirTime, _c14 as ArrivalDelay, _c15 as DepartureDelay, _c16 as Origin, _c17 as Destination, _c18 as Distance, 
    _c19 as TaxiIn, _c20 as TaxiOut, _c21 as Cancelled, _c22 as CancellationCode, _c23 as Diverted, _c24 as CarrierDelay, 
    _c25 as WeatherDelay, _c26 as NASDelay, _c27 as SecurityDelay, _c28 as LateAircraftDelay 
FROM csv.`dbfs:/databricks-datasets/asa/airlines/2008.csv`

- 1단계 작업이 완료되면 표준 'flights' 테이블에는 1년 동안 미국 항공편에 대한 세부 정보가 들어가 있을 것
- 2단계에서는 월별 총 항공편이 가장 높은 상위 20개의 도시를 한 주의 첫째날에 대해 조회

In [71]:
%sql
-- Get top 20 cities with highest monthly total flights on first day of week. & observe the latency! 
-- This query may take over a minute in certain cluster configurations. 
SELECT Month, Origin, count(*) as TotalFlights 
FROM flights
WHERE DayOfWeek = 1 
GROUP BY Month, Origin 
ORDER BY TotalFlights DESC
LIMIT 20;

Month,Origin,TotalFlights
6,ATL,6046
3,ATL,6019
12,ATL,5800
9,ATL,5722
6,ORD,5241
3,ORD,5072
9,ORD,4931
7,ATL,4894
8,ATL,4821
4,ATL,4798


- 2단계가 완료되면 표준 "flights" 테이블을 사용해 latency 시간을 관찰할 수 있음

- 3단계와 4단계에서는 Databricks Delta 테이블로 같은 작업을 수행. 이번에는 쿼리를 실행하기 전에 `ZORDER`와 `OPTIMIZE` 명령을 실행해 데이터가 더 빠른 검색에 최적화되도록 함

In [73]:
%sql
-- 시간 : 6.35 minutes
DROP TABLE IF EXISTS flights;

-- Create a standard table and import US based flights for year 2008
-- USING Clause: Specify "delta" format instead of the standard parquet format
-- PARTITIONED BY clause: Orginize data based on "Origin" column (Originating Airport code).
-- FROM Clause: Import data from a csv file.
CREATE TABLE flights
USING delta
PARTITIONED BY (Origin)
SELECT _c0 as Year, _c1 as Month, _c2 as DayofMonth, _c3 as DayOfWeek, _c4 as DepartureTime, _c5 as CRSDepartureTime, _c6 as ArrivalTime, 
  _c7 as CRSArrivalTime, _c8 as UniqueCarrier, _c9 as FlightNumber, _c10 as TailNumber, _c11 as ActualElapsedTime, _c12 as CRSElapsedTime, 
    _c13 as AirTime, _c14 as ArrivalDelay, _c15 as DepartureDelay, _c16 as Origin, _c17 as Destination, _c18 as Distance, 
    _c19 as TaxiIn, _c20 as TaxiOut, _c21 as Cancelled, _c22 as CancellationCode, _c23 as Diverted, _c24 as CarrierDelay, 
    _c25 as WeatherDelay, _c26 as NASDelay, _c27 as SecurityDelay, _c28 as LateAircraftDelay 
FROM csv.`dbfs:/databricks-datasets/asa/airlines/2008.csv`;

OPTIMIZE 는 파일을 통합하고 더 빠른 검색을 위해 각 파티션 아래에 DayofWeek 별로 Databricks Delta 테이블 데이터를 정렬함

In [75]:
%sql
-- 시간 : 17.17 minutes
OPTIMIZE flights ZORDER BY (DayofWeek);

path,metrics
null,"List(300, 2308, List(8665, 7000929, 384090.0, 300, 115227124), List(6345, 1141360, 62761.0, 2308, 144854578), 0, List(minCubeSize(107374182400), List(0, 0), List(2312, 144883582), 0, List(2308, 144854578), 0), 1)"


- 4단계 : 2단계의 쿼리를 다시 실행하고 latency 시간을 관찰

In [77]:
%sql
-- 시간 : 30.97 seconds
-- Get top 20 cities with highest monthly total flights on first day of week. & observe the latency! 
-- This query may take over a minute in certain cluster configurations. 
SELECT Month, Origin, count(*) as TotalFlights 
FROM flights
WHERE DayOfWeek = 1 
GROUP BY Month, Origin 
ORDER BY TotalFlights DESC
LIMIT 20;

Month,Origin,TotalFlights
6,ATL,6046
3,ATL,6019
12,ATL,5800
9,ATL,5722
6,ORD,5241
3,ORD,5072
9,ORD,4931
7,ATL,4894
8,ATL,4821
4,ATL,4798


Databricks Delta 테이블에 대한 쿼리는 `OPTIMIZE` 가 실행된 후 훨씬 빠르게 실행됨을 알 수 있음. 쿼리 실행 속도는 실행중인 클러스터의 구성에 따라 달라질 수 있지만 표준 테이블에 비해서 5~10배 더 빨라진 것을 알 수 있음